In [37]:
import fitz

def read_pdf_mupdf(file_path):
    doc = fitz.open(file_path)
    text = ''
    for page_number in range(doc.page_count):
        page = doc[page_number]
        text += page.get_text()
    return text

pdf_path = r'premchand.pdf'
pdf_text = read_pdf_mupdf(pdf_path)
print(pdf_text)

 
 
ELEVEN STORIES BY PREMCHAND 
 
Catastrophe 
 
Vidhwans 
 
   In Banaras District there is a village called Bira in  which an old, childless 
widow used to live. She was a  Gond woman named Bhungi and she didn't 
own either a scrap of land or a house to live in. Her only source of livelihood 
was a parching oven. The village folk customarily have one meal a day of 
parched grains, so there was always a crowd around Bhungi's oven. 
Whatever grain she was paid for parching she would grind or fry and eat it. 
She slept in a corner of the same little shack that sheltered the oven. As soon 
as it was light she'd get up and go out to gather dry leaves from all around to 
make her fire. She would stack the leaves right next to the oven, and after 
twelve, light the fire. But on the days when she had to parch grain for Pandit 
Udaybhan Pandey, the owner of the village, she went to bed hungry. She 
was obliged to work without pay for Pandit Udaybhan Pandey She also had 
to fetch water for hi

In [38]:
import pandas as pd
import tiktoken

In [39]:
embedding_encoding = "cl100k_base"
encoding = tiktoken.get_encoding(embedding_encoding)

In [40]:
def tokens(text):
    return len(encoding.encode(text)) 
def split_text_by_tokens(pdf_text, max_tokens=4000):
    text_segments = []
    current_segment = ""

    for line in pdf_text.splitlines():
        if tokens(current_segment + line) <= max_tokens:
            current_segment += line + "\n"
        else:
            text_segments.append(current_segment)
            current_segment = line + "\n"

    if current_segment:
        text_segments.append(current_segment.strip())

    return text_segments

In [41]:
text_segments = split_text_by_tokens(pdf_text) 

In [42]:
import pandas as pd

df = pd.DataFrame(text_segments, columns=["text"])
df.head()

,text
0,\n \nELEVEN STORIES BY PREMCHAND \n \nCatastr...
1,"mahatmas. Now, when they arrive at some well-t..."
2,Buddhu had no opportunity to go to look after ...
3,devastating his field and Halku went on sittin...
4,"heels with the plow. Plough-share, rope, yoke,..."


In [43]:
df['tokens'] = df['text'].apply(lambda x : encoding.encode(x))
df['token_cnt'] = df['tokens'].apply(lambda x : len(x))
df.head()

,text,tokens,token_cnt
0,\n \nELEVEN STORIES BY PREMCHAND \n \nCatastr...,"[33006, 36, 877, 42122, 4015, 43837, 7866, 216...",3989
1,"mahatmas. Now, when they arrive at some well-t...","[75219, 266, 7044, 13, 4800, 11, 994, 814, 177...",3987
2,Buddhu had no opportunity to go to look after ...,"[33, 8512, 17156, 1047, 912, 6776, 311, 733, 3...",3992
3,devastating his field and Halku went on sittin...,"[3667, 561, 1113, 813, 2115, 323, 473, 1727, 8...",3993
4,"heels with the plow. Plough-share, rope, yoke,...","[73610, 449, 279, 628, 363, 13, 1856, 1409, 60...",3989


In [44]:
df['token_cnt'].sum()

28912

In [45]:
from openai import OpenAI
import varkey
client = OpenAI(api_key= varkey.api_key1)

def create_emb(tokens):
    return client.embeddings.create(input=tokens, model="text-embedding-3-large").data[0].embedding

In [46]:
df['embedding'] = df['tokens'].apply(lambda x : create_emb(x))

In [47]:
df.head()

,text,tokens,token_cnt,embedding
0,\n \nELEVEN STORIES BY PREMCHAND \n \nCatastr...,"[33006, 36, 877, 42122, 4015, 43837, 7866, 216...",3989,"[0.018461966887116432, -0.035953253507614136, ..."
1,"mahatmas. Now, when they arrive at some well-t...","[75219, 266, 7044, 13, 4800, 11, 994, 814, 177...",3987,"[0.04438115656375885, -0.06700099259614944, 0...."
2,Buddhu had no opportunity to go to look after ...,"[33, 8512, 17156, 1047, 912, 6776, 311, 733, 3...",3992,"[0.02942752279341221, -0.05666983500123024, 0...."
3,devastating his field and Halku went on sittin...,"[3667, 561, 1113, 813, 2115, 323, 473, 1727, 8...",3993,"[0.0466248095035553, -0.09093517810106277, 0.0..."
4,"heels with the plow. Plough-share, rope, yoke,...","[73610, 449, 279, 628, 363, 13, 1856, 1409, 60...",3989,"[0.04742599278688431, -0.034924816340208054, 0..."


In [35]:
from pinecone import Pinecone
import varkey1
pc = Pinecone(api_key= varkey1.api_key2)
index = pc.Index("book")

In [36]:
def upload_embeddings(emb, text, i):
    index.upsert(vectors=[{"id": str(i), "values": emb, "metadata":{"text":text}}])

for i, row in df.iterrows():
    upload_embeddings(row['embedding'], row['text'], i)